# All imports and inits

In [27]:
import pandas as pd
import glob
import concurrent.futures
import gradio as gr
import tkinter as tk
import streamlit as st
import numpy as np
import textwrap
import PyPDF2
import requests
import os
import pinecone
import time

from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from dotenv import load_dotenv
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from tkinter import scrolledtext, messagebox
from transformers import AutoModel, AutoTokenizer
from typing import List, Tuple
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec, CloudProvider, AwsRegion, VectorType


load_dotenv()

DATA_PATH = os.getenv("DATA_PATH")
PINECONE_API = os.getenv("PINECONE_API")
# PINECONE_ENV = os.getenv("PINECONE_ENV")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
GROQ_CHAT_URL = os.getenv("GROQ_CHAT_URL")

NVIDIA_API = os.getenv("NVIDIA_API")
NVIDIA_BASE_URL = os.getenv("NVIDIA_BASE_URL")

# Configure headers for Groq API requests
GROQ_HEADERS = {
    "Authorization": f"Bearer {GROQ_API_KEY}",
    "Content-Type": "application/json",
}
# LLM_MODEL = "llama3-70b-8192"
LLM_MODEL = "llama-3.3-70b-versatile"


# NVidia Embedding import
client = OpenAI(
    api_key=NVIDIA_API,
    base_url=NVIDIA_BASE_URL,
)

"""
Input:
    - Context window: 128K
Ouput:
    - Output Max Tokens: 32,768

"""


def track_time(func):
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print(f"[Time Tracker] `{func.__name__}` took {end - start:.4f} seconds")
        return result

    return wrapper


# # EMBEDDING_MODEL = "llama3-405b-8192-embed"

# vo = voyageai.Client()


# Init Pinecone

In [28]:
pc = Pinecone(api_key=PINECONE_API)
print(PINECONE_API)


pcsk_2zBTsW_U5bfHyCathSYufEN1wYf1t3v9jtSX9T94Zqk3z8h7djMZFM324gKjpeoKKhkr4q


# Embedding Function



In [29]:
# Connect to the index
index = pc.Index("ai-coach")
# # index = pc.Index("ahsan-400pg-pdf-doc-test")

# embedding_model = AutoModel.from_pretrained(
#     'jinaai/jina-embeddings-v2-base-en', trust_remote_code=True)


# # Function to generate embeddings without tokenization
# def get_embedding(data):
#     embeddings = embedding_model.encode(data).tolist()
#     return embeddings


@track_time
def get_embedding(text="None"):
    response = client.embeddings.create(
        input=text,
        model="nvidia/nv-embed-v1",
        encoding_format="float",
        extra_body={"input_type": "query", "truncate": "NONE"},
    )

    # print(response.data[0].embedding)
    # print(count_tokens(response.data[0].embedding))
    return response.data[0].embedding


get_embedding("None")


[Time Tracker] `get_embedding` took 1.2503 seconds


[-0.01506805419921875,
 -0.01165008544921875,
 0.02081298828125,
 0.0252227783203125,
 -0.0198211669921875,
 0.011077880859375,
 0.00782012939453125,
 0.037872314453125,
 -0.00841522216796875,
 0.00612640380859375,
 -0.0089569091796875,
 0.01071929931640625,
 0.00724029541015625,
 0.0088653564453125,
 0.004428863525390625,
 -0.00255584716796875,
 0.0171356201171875,
 -0.0015773773193359375,
 0.0038318634033203125,
 -0.01238250732421875,
 0.020904541015625,
 -0.00855255126953125,
 0.0176849365234375,
 0.0019550323486328125,
 -0.0012798309326171875,
 0.0123748779296875,
 -0.001995086669921875,
 0.006771087646484375,
 0.032135009765625,
 -0.00537872314453125,
 -0.02215576171875,
 0.007434844970703125,
 -0.014984130859375,
 -0.0027523040771484375,
 -0.008514404296875,
 -0.00858306884765625,
 -0.005031585693359375,
 -0.00081634521484375,
 -0.010833740234375,
 -0.0025196075439453125,
 0.008209228515625,
 -0.0005240440368652344,
 0.017333984375,
 -0.0037689208984375,
 -0.0098876953125,
 0.011

# Query Pinecone


In [33]:
# Function to query Pinecone index using embeddings
@track_time
def query_pinecone(embedding):
    # Use keyword arguments to pass the embedding and other parameters
    result = index.query(vector=embedding, top_k=320, include_metadata=True)
    return result["matches"]


# Query Groq Inference

In [34]:
from groq import Groq
# Function to query Groq LLM
# def query_groq(prompt: str) -> str:
#     response = requests.post(
#         GROQ_CHAT_URL,
#         headers=GROQ_HEADERS,
#         json={
#             "model": LLM_MODEL,
#             "messages": [{"role": "user", "content": prompt}],
#             "temperature": 0.5,
#             # "max_tokens": 8192  # max from groq website
#         },
#     )

#     if response.status_code != 200:
#         raise Exception(f"Error querying Groq: {response.text}")

#     return response.json()["choices"][0]["message"]["content"]


# def query_groq(prompt: str) -> str:
#     client = Groq()
#     completion = client.chat.completions.create(
#         messages=[{"role": "user", "content": prompt}],
#         # Change model to compound-beta to use agentic tooling
#         # model: "llama-3.3-70b-versatile",
#         model=LLM_MODEL,
#     )
#     # print(completion.choices[0].message.content)
#     return completion.choices[0].message.content


@track_time
def query_groq(prompt: str) -> str:
    client = Groq()
    completion = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model=LLM_MODEL,
        temperature=0.5,  # Set temperature to 0.5
    )
    return completion.choices[0]


query_groq("Hello")
# Print all tool calls
# print(completion.choices[0].message.executed_tools)


# Tokenizer to count number of tokens


@track_time
def count_tokens(text: str) -> int:
    tokenizer = AutoTokenizer.from_pretrained("jinaai/jina-embeddings-v2-base-en")
    # Encode the text into tokens
    tokens = tokenizer.encode(text)
    return len(tokens)


[Time Tracker] `query_groq` took 1.5638 seconds


# Process User Query

### Gradio GUI TEST

In [37]:
# ------------------------------------------- WORKING 3 Enter key submits user query -------------------------------------------
# Initialize empty conversation history (list of tuples)
conversation_history = []


@track_time
def process_user_query(user_query: str, conversation_history: list):
    print(f"User Query Tokens: {count_tokens(user_query)}")

    # Generate embedding and get relevant context
    embedding = get_embedding(user_query)
    relevant_chunks = query_pinecone(embedding)
    context = "\n".join(chunk["metadata"]["text"] for chunk in relevant_chunks)
    #print("CONTEXT:", context)

    # Format conversation history for the prompt
    history_str = "\n".join(
        f"User: {user}\nCoach: {response}" for user, response in conversation_history
    )

    # Create structured prompt
    prompt = f"""You are an expert, knowledgeable, and friendly coach. Follow these guidelines carefully:

    1. Provide clear, step-by-step explanations to ensure deep understanding.
    2. Use chain-of-thought reasoning to thoroughly evaluate the provided context before responding.
    3. Ask guiding questions to encourage critical thinking.
    4. Adapt your explanation to match the student's knowledge level.
    5. Strictly use terminologies provided in the given context.
    6. Provide short, ideal examples (2-3) to illustrate your points clearly.
    7. Only answer based on the provided context—do not speculate or include external information.
    8. Explicitly cite the sources from the context in your responses such as urls, video timestamps , links , resources , ebook names etc.
    9. Perform sentiment analysis based on conversation history and user queries to adapt your responses empathetically and effectively.

    Context from learning materials:
    {context}

    Conversation history:
    {history_str}

    New student question:
    "{user_query}"

    Provide a helpful, structured response that meets the above criteria.

    (Note: The following examples are only provided for your reference to demonstrate an effective response format):

    Question: How long will the average externship take to complete?
    Answer: 125 surgical cases – typically 6 months to 1 year.

    Question: What should I focus on when studying anatomy, physiology, and medical terminology?
    Answer: Focus specifically on content related to surgical procedures, emphasizing body systems, terminology, and physiological functions most relevant to surgery.

    Question: What’s the best way to study and memorize surgical instrumentation?
    Answer: First, understand the National Center for Competency Testing (NCCT) exam expectations regarding instruments. Refer to official NCCT guidelines and utilize platforms such as Quizlet and ProProfs for visual memorization and repetition.

    Provide a thoughtful and contextually accurate response now:"""

    # --------------------- Reasoning Prompt --------------------------------------------------------------
    # prompt = f"""You are an expert, knowledgeable, and friendly coach. Follow this structured response framework:

    # ### Response Requirements
    # 1. **Reasoning Process**:
    # - Analyze the question against provided context
    # - Identify relevant context snippets with source citations
    # - Perform sentiment analysis on conversation history

    # 2. **Response Format**:
    # - [Understanding] Paraphrase the question to confirm comprehension
    # - [Relevant Context] Cite exact source material with location references
    # - [Step-by-Step Explanation] Break down concepts using chain-of-thought
    # - [Examples] Provide 2-3 ideal examples from context
    # - [Guiding Questions] Pose 1-2 reflective questions to deepen learning
    # - [Summary] Concise answer reiterating key points

    # 3. **Style Guidelines**:
    # - Use terminology strictly from context
    # - Adapt complexity to student's history
    # - Maintain empathetic tone based on sentiment analysis

    # ### Context Materials:
    # {context}

    # ### Conversation History:
    # {history_str}

    # ### New Question:
    # "{user_query}"

    # ### Your Response:
    # [Understanding] First, let me clarify what you're asking...
    # [Relevant Context] According to [Source X, Section Y]...
    # [Step-by-Step Explanation] The process works as follows:
    # 1. First concept...
    # 2. Second concept...
    # 3. Practical application...
    # [Examples] For instance:
    # - Example 1...
    # - Example 2...
    # [Guiding Questions] Have you considered...? How might this apply to...?
    # [Summary] To recap the key points..."""

    # Get LLM response
    groq_response = query_groq(prompt)
    print(f"Response Toke   ns: {count_tokens(groq_response.message.content)}")

    # Return updated history with new interaction
    return conversation_history + [(user_query, groq_response.message.content)]


# Gradio Interface
with gr.Blocks() as interface:
    gr.Markdown("# 🧑‍🏫 AI Coaching Assistant")
    gr.Markdown("Welcome! I'm here to help you learn. Type your question below.")

    # State management
    chat_history = gr.State(conversation_history)

    with gr.Row():
        chatbot = gr.Chatbot(height=500)
        with gr.Column(scale=0.5):
            context_display = gr.Textbox(label="Relevant Context", interactive=False)

    user_input = gr.Textbox(label="Your Question", placeholder="Type here...")

    with gr.Row():
        submit_btn = gr.Button("Submit", variant="primary")
        undo_btn = gr.Button("Undo Last")
        clear_btn = gr.Button("Clear History")

    def handle_submit(user_input, history):
        if not user_input.strip():
            return gr.update(), history, ""

        # Process query and update history
        new_history = process_user_query(user_input, history)

        # Get latest context for display
        latest_context = "\n".join(
            [
                chunk["metadata"]["text"]
                for chunk in query_pinecone(get_embedding(user_input))
            ][:]
        )  # Show top 3 context snippets

        return "", new_history, latest_context

    # Component interactions
    submit_btn.click(
        handle_submit,
        [user_input, chat_history],
        [user_input, chat_history, context_display],
    ).then(lambda x: x, [chat_history], [chatbot])

    # Add submit on Enter key press
    user_input.submit(
        handle_submit,
        [user_input, chat_history],
        [user_input, chat_history, context_display],
    ).then(lambda x: x, [chat_history], [chatbot])

    undo_btn.click(
        lambda history: history[:-1] if history else [], [chat_history], [chat_history]
    ).then(lambda x: x, [chat_history], [chatbot])

    clear_btn.click(lambda: [], None, [chat_history]).then(
        lambda: ([], ""), None, [chatbot, context_display]
    )

interface.launch(share=True)


C:\Users\ABC\AppData\Local\Temp\ipykernel_11732\772650616.py:119: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=500)
D:\Disrupt Labs\AI Coach Project\Rag-pipelines-experiments\env\Lib\site-packages\gradio\layouts\column.py:55: UserWarning: 'scale' value should be an integer. Using 0.5 will cause issues.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7869
* Running on public URL: https://c7b9089a6e3046f1ef.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


[Time Tracker] `count_tokens` took 0.4154 seconds
User Query Tokens: 4
[Time Tracker] `get_embedding` took 1.4235 seconds
[Time Tracker] `query_pinecone` took 13.7592 seconds
[Time Tracker] `query_groq` took 6.6848 seconds
[Time Tracker] `count_tokens` took 0.5924 seconds
Response Toke   ns: 593
[Time Tracker] `process_user_query` took 22.8821 seconds
[Time Tracker] `get_embedding` took 1.5100 seconds
[Time Tracker] `query_pinecone` took 8.7653 seconds
